In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('mlboot_dataset/mlboot_train_answers.tsv.gz', delimiter='\t', compression='gzip', index_col=0)
test = pd.read_csv('mlboot_dataset/mlboot_test.tsv.gz', delimiter='\t', compression='gzip')

In [3]:
train.target.value_counts()

0    406542
1     21452
Name: target, dtype: int64

In [4]:
df = pd.read_csv('mlboot_dataset/mlboot_data.tsv.gz', delimiter='\t', compression='gzip', header=None, 
                      names=['cuid', 'cat_feature', 'counter1', 'counter2', 'counter3', 'dt_diff'])

In [5]:
df.head()

,cuid,cat_feature,counter1,counter2,counter3,dt_diff
0,00000d2994b6df9239901389031acaac,5,"{""809001"":2,""848545"":2,""565828"":1,""490363"":1}","{""85789"":1,""238490"":1,""32285"":1,""103987"":1,""16...",{},39
1,00000d2994b6df9239901389031acaac,5,"{""809001"":2,""1160689"":1,""14018"":1,""843995"":1,""...","{""9629"":1,""894555"":1,""2812609"":1,""15310"":1,""48...",{},43
2,00000d2994b6df9239901389031acaac,5,"{""536657"":1,""1723154"":1,""948501"":1,""1159583"":1...","{""129137"":1,""14056"":1,""9763"":1,""44215"":1,""3671...",{},45
3,00000d2994b6df9239901389031acaac,5,"{""1676126"":1,""1321640"":1,""1198"":1,""809001"":2,""...","{""153774"":1,""2127"":1,""158967"":1}",{},46
4,00000d2994b6df9239901389031acaac,5,"{""1125045"":2,""1596919"":1,""1857574"":1,""282284"":...","{""2979"":1,""30581"":1,""7296"":1,""127256"":1,""18803...",{},48


In [6]:
import json
import itertools


In [7]:
from typing import Dict, List, Iterable, Tuple
def dict_to_dense(dict_row: str) -> List[int]:
    s = []
    for k,v in json.loads(dict_row).items():
        s.extend(itertools.repeat(int(k), v))
    return s

In [8]:
def extract_from_row(row: pd.Series) -> Tuple[str, str, str]:
# def extract_from_row(row: pd.Series):

    category = str(row.cat_feature)
    day = str(row.dt_diff)
    
    feat1 = dict_to_dense(row.counter1)
    feat2 = dict_to_dense(row.counter2)
    feat3 = dict_to_dense(row.counter3)
    counter = list(itertools.chain(feat1, feat2, feat3))
    cc = ' '.join(str(v) for v in counter)
    return cc, day, category

In [103]:
test_dataset = dataset.head(100)

### Основная идея: "Развернуть" данные в плоский вид, чтобы можно было на основе этих данных составить sparse-матрицы фич

In [9]:
from tqdm import tqdm

In [13]:
output_train = 'sparse/train_counter.csv'
output_train_cat = 'sparse/train_cat.csv'
output_train_day = 'sparse/train_day.csv'
output_train_cuid = 'sparse/train_cuid.csv'
output_train_labels = 'sparse/train_labels.csv'

output_test = 'sparse/test_counter.csv'
output_test_cat = 'sparse/test_cat.csv'
output_test_day = 'sparse/test_day.csv'
output_test_cuid = 'sparse/test_cuid.csv'

In [11]:
df.shape[0]

19528597

In [14]:
prev_cuid = ''
prev_cuid_in_train = None

with open(output_train, 'w') as dataset, \
    open(output_train_cat, 'w') as dataset_cat, \
    open(output_train_day, 'w') as dataset_day, \
    open(output_train_cuid, 'w') as dataset_cuid, \
    open(output_train_labels, 'w') as dataset_labels, \
    open(output_test, 'w') as test, \
    open(output_test_cat, 'w') as test_cat, \
    open(output_test_day, 'w') as test_day, \
    open(output_test_cuid, 'w') as test_cuid:

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        cuid = row["cuid"]
        
        counter, day, cat_featrue = extract_from_row(row)
        
        if cuid != prev_cuid:
            
            if prev_cuid_in_train:
                dataset.write(counter_full + '\n')
                dataset_cat.write(cat_full + '\n')
                dataset_day.write(day_full + '\n')
                dataset_labels.write(str(label) + '\n')
            elif prev_cuid_in_train is not None:
                test.write(counter_full + '\n')
                test_cat.write(cat_full + '\n')
                test_day.write(day_full + '\n')
            
            cat_full = cat_feature
            counter_full = counter
            dayn_full = day
            
            cuid_in_train = cuid in train.index
            if cuid_in_train:
                label = train['target'][cuid]
                dataset_cuid.write(f'{cuid}\n')
            else:
                test_cuid.write(f'{cuid}\n')
            
            prev_cuid = cuid
            prev_cuid_in_train = cuid_in_train
        else:
            cat_full += ' ' + cat_feature
            counter_full += ' ' + counter
            day_full += ' ' + day
    
    # Записать последнюю строку в файл
    if prev_cuid_in_train:
        dataset.write(counter_full + '\n')
        dataset_cat.write(cat_full + '\n')
        dataset_day.write(day_full + '\n')
        dataset_labels.write(str(label) + '\n')
    else:
        test.write(counter_full + '\n')
        test_cat.write(cat_full + '\n')
        test_day.write(day_full + '\n')
    
    # Последний cuid
    if cuid in train.index:
        dataset_cuid.write(f'{cuid}\n')
    else:
        test_cuid.write(f'{cuid}\n')
    

100%|██████████| 19528597/19528597 [1:07:27<00:00, 4824.54it/s]


### Создадим теперь sparse матрицы на основе этих датасетов

In [23]:
import gc
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy import sparse


#### В данном соревновании вполне можно использовать TF-IDF для каунтеров, и каунтвекторайзер для остальных фичей

### Counters features

In [24]:
tf_idf = TfidfVectorizer(min_df=3, max_df=0.4, sublinear_tf=True)

In [38]:
train_counter = pd.read_csv("sparse/train_counter.csv", names=["counters"])

In [39]:
train_counter.head()

,counters
0,809001 809001 848545 848545 565828 490363 8578...
1,809001 809001 1171980 1166289 618566 809001 80...
2,581893 581893 834560 345885 16744 6366 42224 1...
3,1714680 1961140 1153101 176510 4816 93716 2268...
4,990342 389051 1202056 834293 47006 724959 1283...


In [40]:
%%time
counter_matrix = tf_idf.fit_transform(train_counter.counters)

CPU times: user 12min 56s, sys: 17.3 s, total: 13min 14s
Wall time: 13min 11s


In [41]:
sparse.save_npz('sparse/sparse_counter.npz', counter_matrix)

In [42]:
test_counter = pd.read_csv("sparse/test_counter.csv", names=["counters"])

In [69]:
counter_matrix = tf_idf.transform(test_counter.counters)

In [70]:
sparse.save_npz('sparse/test_sparse_counter.npz', counter_matrix)

### Categorical features

In [49]:
count_vect = CountVectorizer(analyzer='char')

In [51]:
train_categorical = pd.read_csv("sparse/train_cat.csv", names=['category'])

In [53]:
category_matrix = count_vect.fit_transform(train_categorical.category)

In [54]:
sparse.save_npz('sparse/sparse_category.npz', category_matrix)

In [55]:
test_categorical = pd.read_csv("sparse/test_cat.csv", names=['category'])

In [57]:
category_matrix = count_vect.transform(test_categorical.category)

In [58]:
sparse.save_npz('sparse/test_sparse_category.npz', category_matrix)

### days_diff features

In [60]:
train_day = pd.read_csv("sparse/train_day.csv", names=['days'])

In [62]:
count_vect = CountVectorizer()

In [63]:
day_matrix = count_vect.fit_transform(train_day.days)
sparse.save_npz('sparse/sparse_days.npz', day_matrix)
del day_matrix
gc.collect()
test_day = pd.read_csv("sparse/test_day.csv", names=['days'])

day_matrix = count_vect.transform(test_day.days)
sparse.save_npz('sparse/test_sparse_days.npz', day_matrix)

## Concat all sparse

In [65]:
day_matrix = sparse.load_npz("sparse/sparse_days.npz")
category_matrix = sparse.load_npz("sparse/sparse_category.npz")
counter_matrix = sparse.load_npz("sparse/sparse_counter.npz")

In [66]:
train_matrix = sparse.hstack([day_matrix, category_matrix, counter_matrix])
sparse.save_npz('sparse/train_matrix.npz', train_matrix)

In [71]:
day_matrix = sparse.load_npz("sparse/test_sparse_days.npz")
category_matrix = sparse.load_npz("sparse/test_sparse_category.npz")
counter_matrix = sparse.load_npz("sparse/test_sparse_counter.npz")

In [72]:
test_matrix = sparse.hstack([day_matrix, category_matrix, counter_matrix])
sparse.save_npz('sparse/test_matrix.npz', test_matrix)